In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train=pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

**Let's look at the Data.**

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train['target'].value_counts()

In [ ]:
y = train['target']

In [ ]:
colstodrop = ["ID_code","target"]

In [ ]:
data = train.drop(colstodrop,axis = 1)

In [ ]:
len(train.columns)

In [ ]:
data.head()

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=64)
svd.fit(data)

In [ ]:
svddf = pd.DataFrame(svd.transform(data))

In [ ]:
svddf.head()

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 2)
kmeans.fit(data)

In [ ]:
clusters = kmeans.labels_.tolist()

In [ ]:
data1 = pd.concat([data,pd.DataFrame(clusters)],axis=1)

Light GBM is a gradient boosting framework that uses tree based learning algorithm.

LightGBM uses the** leaf-wise tree growth algorithm**, while many other popular tools use depth-wise tree growth.

To get good results using a leaf-wise tree, these are some important parameters:

**num_leaves**. This is the main parameter to control the complexity of the tree model. 
**min_data_in_leaf.** This is a very important parameter to prevent over-fitting in a leaf-wise tree. Its optimal value depends on the number of training samples and num_leaves. Setting it to a large value can avoid growing too deep a tree, but may cause under-fitting. In practice, setting it to hundreds or thousands is enough for a large dataset.
max_depth. We  can use max_depth to limit the tree depth explicitly.

In [ ]:
from sklearn.model_selection import train_test_split
dev_X, val_X, dev_y, val_y = train_test_split(data,y, test_size = 0.2, random_state = 42)

In [ ]:
params = {
    'objective' :'binary',
    'learning_rate' : 0.02,
    'num_leaves' : 76,
    'feature_fraction': 0.64, 
    'bagging_fraction': 0.8, 
    'bagging_freq':1,
    'boosting_type' : 'gbdt',
    'metric': 'auc'
}

In [ ]:
testd = test.drop("ID_code",axis=1)

In [ ]:
import lightgbm as lgb
def run_lgb(train_X, train_y, val_X, val_y):
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 15000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=50, 
                      evals_result=evals_result)
    
    pred_test_y = model.predict(testd, num_iteration=model.best_iteration)
    return pred_test_y, model, evals_result

# Training the model #
pred_test, model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y)

In [ ]:
lgbsubmission = pd.DataFrame({"ID_code": test["ID_code"], "target": pred_test })
lgbsubmission.to_csv('lgbsubmission.csv', index=False)